In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data : German elections

We have two files on regional elections in Germany (2005 and 2009), giving for each vote place the ranking of the candidates at the election, with several characteristics of the votants.

The idea could be to train the label ranking on 2005 data, and evaluate the prediction of this one on the 2009 data, to see if the model could be able to predict the results of an election, based on the previous ones.

### Raw datas

The labels a, b, c, d, e correspond to the following political parties in Germany :

• CDU (conservative)

• SPD (center-left)

• FDP (liberal)

• GREEN (center-left)

• LEFT (left-wing)

In [2]:
german_2005 = pd.read_csv("data_new/lr_german_2005.csv", sep=";")
german_2005.head()

,Type,State,Region,Elect..Particptn..2005,Elect..Participtn..2009,Pop..Density,Old.Pop.,Mid.aged.Pop.,Young.Pop.,Children.Pop.,...,Bankruptcies,Unemploy.,Unempl..Youth,lat,lon,a,b,c,d,e
0,Urban,Schleswig-Holstein,North,73.2,67.4,1559.8,24.3,29.1,11.7,15.4,...,203.6,13.0,12.2,54.793149,9.445471,2,1,4,3,5
1,Urban,Schleswig-Holstein,North,77.0,71.7,2008.3,25.5,30.9,10.8,14.5,...,150.2,10.9,8.5,54.322395,10.121557,2,1,4,3,5
2,Urban,Schleswig-Holstein,North,73.1,68.3,979.5,25.9,26.6,8.8,15.7,...,116.2,12.3,12.9,53.865010,10.681037,2,1,4,3,5
3,Urban,Schleswig-Holstein,North,72.5,65.4,1073.5,26.2,25.2,8.9,17.8,...,131.5,11.3,12.8,54.072245,9.990648,2,1,3,4,5
4,Rural,Schleswig-Holstein,North,77.3,71.1,94.7,27.4,23.6,7.7,18.3,...,86.7,9.7,11.8,54.195788,9.101376,1,2,3,4,5


In [4]:
german_2009 = pd.read_csv("data_new/lr_german_2009.csv", sep=";")
german_2009.head()

,Type,State,Region,Elect..Particptn..2005,Elect..Participtn..2009,Elderly.pop.,Old.Pop.,Mid.aged.Pop.,Young.Pop.,Children.Pop.,...,Bankruptcies,Unemploy.,Unempl..Youth,lat,lon,a,b,c,d,e
0,Urban,Schleswig-Holstein,North,73.2,67.4,19.6,24.3,29.1,11.7,15.4,...,203.6,13.0,12.2,54.793149,9.445471,2,1,4,3,5
1,Urban,Schleswig-Holstein,North,77.0,71.7,18.5,25.5,30.9,10.8,14.5,...,150.2,10.9,8.5,54.322395,10.121557,2,1,4,3,5
2,Urban,Schleswig-Holstein,North,73.1,68.3,23.1,25.9,26.6,8.8,15.7,...,116.2,12.3,12.9,53.865010,10.681037,2,1,3,4,5
3,Urban,Schleswig-Holstein,North,72.5,65.4,21.9,26.2,25.2,8.9,17.8,...,131.5,11.3,12.8,54.072245,9.990648,2,1,3,4,5
4,Rural,Schleswig-Holstein,North,77.3,71.1,23.1,27.4,23.6,7.7,18.3,...,86.7,9.7,11.8,54.195788,9.101376,1,2,3,4,5


In [10]:
german_2005.columns

Index(['Type', 'State', 'Region', 'Elect..Particptn..2005',
       'Elect..Participtn..2009', 'Pop..Density', 'Old.Pop.', 'Mid.aged.Pop.',
       'Young.Pop.', 'Children.Pop.', 'Highschool.degree', 'No.school.degree',
       'Registrd.Web.Doms', 'Business.Regstr.', 'Income', 'GDP.growth.2009',
       'GDP.growth.2008', 'Workforce', 'X.cars', 'Agricult..workf.',
       'Prod..workf.', 'Manufac..Workf.', 'Constr..workf.', 'Service.workf.',
       'Trade.workf.', 'Finance.workf.', 'Pub..serv..workf.', 'Bankruptcies',
       'Unemploy.', 'Unempl..Youth', 'lat', 'lon', 'a', 'b', 'c', 'd', 'e'],
      dtype='object')

#### check consistency of the two files

- data is measured exactly on the same vote places

In [11]:
print(len(german_2005), len(german_2009))
print(np.all(german_2005['lat'] == german_2009['lat']))
print(np.all(german_2005['lon'] == german_2009['lon']))
print(np.all(german_2005['State'] == german_2009['State']))
print(np.all(german_2005['Region'] == german_2009['Region']))

412 412
True
True
True
True


- only one column is differing : density of population in 2005 instead of elderly population in 2009
- for the approach, drop the participation of the not considered year (keep only participation in 2005 for 2005, 2009 for 2009)

In [16]:
print(german_2005.columns == german_2009.columns)
print(german_2005.columns)
print(german_2009.columns)

[ True  True  True  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True]
Index(['Type', 'State', 'Region', 'Elect..Particptn..2005',
       'Elect..Participtn..2009', 'Pop..Density', 'Old.Pop.', 'Mid.aged.Pop.',
       'Young.Pop.', 'Children.Pop.', 'Highschool.degree', 'No.school.degree',
       'Registrd.Web.Doms', 'Business.Regstr.', 'Income', 'GDP.growth.2009',
       'GDP.growth.2008', 'Workforce', 'X.cars', 'Agricult..workf.',
       'Prod..workf.', 'Manufac..Workf.', 'Constr..workf.', 'Service.workf.',
       'Trade.workf.', 'Finance.workf.', 'Pub..serv..workf.', 'Bankruptcies',
       'Unemploy.', 'Unempl..Youth', 'lat', 'lon', 'a', 'b', 'c', 'd', 'e'],
      dtype='object')
Index(['Type', 'State', 'Region', 'Elect..Particptn..2005',
       'Elect..Participtn..2009', 'Elderly.pop.', 'Old.Pop.', 'Mid.aged.Pop.',
       'Young.P

### adapting the data to a compatible format

- the ranking variable should be at the format 'a>b>c>d>e'

In [17]:
# fct to transform a list of rankings in string
def list_to_str(list_rank):
    return '>'.join([str(elem) for elem in list_rank])

In [18]:
def write_rank(data):
    data['ranking'] = ""
    for i in range(len(data)):
        data['ranking'][i] = [0, 0, 0, 0, 0]
        for item in ["a", "b", "c", "d", "e"]:
            nb = data.iloc[i][item]
            data.iloc[i]['ranking'][nb-1] = item
    data['ranking'] = data['ranking'].apply(list_to_str)
    return data

In [19]:
german_2005_ranked = write_rank(german_2005)
german_2005_ranked

C:\Users\e32cl\AppData\Local\Temp\ipykernel_36760\1960091903.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ranking'][i] = [0, 0, 0, 0, 0]


,Type,State,Region,Elect..Particptn..2005,Elect..Participtn..2009,Pop..Density,Old.Pop.,Mid.aged.Pop.,Young.Pop.,Children.Pop.,...,Unemploy.,Unempl..Youth,lat,lon,a,b,c,d,e,ranking
0,Urban,Schleswig-Holstein,North,73.2,67.4,1559.8,24.3,29.1,11.7,15.4,...,13.0,12.2,54.793149,9.445471,2,1,4,3,5,b>a>d>c>e
1,Urban,Schleswig-Holstein,North,77.0,71.7,2008.3,25.5,30.9,10.8,14.5,...,10.9,8.5,54.322395,10.121557,2,1,4,3,5,b>a>d>c>e
2,Urban,Schleswig-Holstein,North,73.1,68.3,979.5,25.9,26.6,8.8,15.7,...,12.3,12.9,53.865010,10.681037,2,1,4,3,5,b>a>d>c>e
3,Urban,Schleswig-Holstein,North,72.5,65.4,1073.5,26.2,25.2,8.9,17.8,...,11.3,12.8,54.072245,9.990648,2,1,3,4,5,b>a>c>d>e
4,Rural,Schleswig-Holstein,North,77.3,71.1,94.7,27.4,23.6,7.7,18.3,...,9.7,11.8,54.195788,9.101376,1,2,3,4,5,a>b>c>d>e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Rural,Thuringia,East,75.7,64.3,114.3,31.2,23.7,7.7,11.5,...,10.8,10.5,50.647047,11.296807,3,1,4,5,2,b>e>a>c>d
408,Rural,Thuringia,East,78.1,67.7,107.0,32.0,25.0,7.8,12.9,...,9.5,9.7,50.912925,11.787938,3,1,4,5,2,b>e>a>c>d
409,Rural,Thuringia,East,76.9,66.3,77.2,31.6,24.2,7.9,12.7,...,9.7,8.5,50.589275,11.728708,3,1,4,5,2,b>e>a>c>d
410,Rural,Thuringia,East,77.7,66.7,129.2,32.2,23.1,7.1,12.0,...,11.4,10.0,50.654013,12.172110,2,1,4,5,3,b>a>e>c>d


In [20]:
german_2009_ranked = write_rank(german_2009)
german_2009_ranked

C:\Users\e32cl\AppData\Local\Temp\ipykernel_36760\1960091903.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ranking'][i] = [0, 0, 0, 0, 0]


,Type,State,Region,Elect..Particptn..2005,Elect..Participtn..2009,Elderly.pop.,Old.Pop.,Mid.aged.Pop.,Young.Pop.,Children.Pop.,...,Unemploy.,Unempl..Youth,lat,lon,a,b,c,d,e,ranking
0,Urban,Schleswig-Holstein,North,73.2,67.4,19.6,24.3,29.1,11.7,15.4,...,13.0,12.2,54.793149,9.445471,2,1,4,3,5,b>a>d>c>e
1,Urban,Schleswig-Holstein,North,77.0,71.7,18.5,25.5,30.9,10.8,14.5,...,10.9,8.5,54.322395,10.121557,2,1,4,3,5,b>a>d>c>e
2,Urban,Schleswig-Holstein,North,73.1,68.3,23.1,25.9,26.6,8.8,15.7,...,12.3,12.9,53.865010,10.681037,2,1,3,4,5,b>a>c>d>e
3,Urban,Schleswig-Holstein,North,72.5,65.4,21.9,26.2,25.2,8.9,17.8,...,11.3,12.8,54.072245,9.990648,2,1,3,4,5,b>a>c>d>e
4,Rural,Schleswig-Holstein,North,77.3,71.1,23.1,27.4,23.6,7.7,18.3,...,9.7,11.8,54.195788,9.101376,1,2,3,4,5,a>b>c>d>e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Rural,Thuringia,East,75.7,64.3,25.8,31.2,23.7,7.7,11.5,...,10.8,10.5,50.647047,11.296807,2,3,4,5,1,e>a>b>c>d
408,Rural,Thuringia,East,78.1,67.7,22.4,32.0,25.0,7.8,12.9,...,9.5,9.7,50.912925,11.787938,1,3,4,5,2,a>e>b>c>d
409,Rural,Thuringia,East,76.9,66.3,23.7,31.6,24.2,7.9,12.7,...,9.7,8.5,50.589275,11.728708,1,3,4,5,2,a>e>b>c>d
410,Rural,Thuringia,East,77.7,66.7,25.4,32.2,23.1,7.1,12.0,...,11.4,10.0,50.654013,12.172110,1,3,4,5,2,a>e>b>c>d


- get dummies for categorical variables (equivalent to one-hot encoding as we don't have not explored categories)

In [30]:
german_2005_encoded = pd.get_dummies(german_2005_ranked.drop(['a', 'b', 'c', 'd', 'e'], axis=1), 
                                  columns = ['Type', 'State', 'Region'], drop_first=True)

In [22]:
# save this version for a first test on the model
german_2005_encoded.to_csv("data_new/german_modif1.txt", index=False, sep=",")

In [27]:
german_2009_encoded = pd.get_dummies(german_2009_ranked.drop(['a', 'b', 'c', 'd', 'e'], axis=1), 
                                  columns = ['Type', 'State', 'Region'], drop_first=True)

In [24]:
# save this version for a first test on the model
german_2009_encoded.to_csv("data_new/german_2009_modif1.txt", index=False, sep=",")

In [28]:
german_2009_encoded.columns

Index(['Elect..Particptn..2005', 'Elect..Participtn..2009', 'Elderly.pop.',
       'Old.Pop.', 'Mid.aged.Pop.', 'Young.Pop.', 'Children.Pop.',
       'Highschool.degree', 'No.school.degree', 'Registrd.Web.Doms',
       'Business.Regstr.', 'Income', 'GDP.growth.2009', 'GDP.growth.2008',
       'Workforce', 'X.cars', 'Agricult..workf.', 'Prod..workf.',
       'Manufac..Workf.', 'Constr..workf.', 'Service.workf.', 'Trade.workf.',
       'Finance.workf.', 'Pub..serv..workf.', 'Bankruptcies', 'Unemploy.',
       'Unempl..Youth', 'lat', 'lon', 'ranking', 'Type_Urban', 'State_Bavaria',
       'State_Berlin', 'State_Brandenburg', 'State_Bremen', 'State_Hamburg',
       'State_Hesse', 'State_Lower Saxony', 'State_Mecklenburg-Vorpommern',
       'State_NRW', 'State_Rhineland-Palatinate', 'State_Saarland',
       'State_Saxony', 'State_Saxony-Anhalt', 'State_Schleswig-Holstein',
       'State_Thuringia', 'Region_North', 'Region_South', 'Region_West'],
      dtype='object')

- drop not common variables at the two dataset
- drop participation at the not correspondind date
- drop coordinates (to avoid overfitting)

In [31]:
german_2005_last = german_2005_encoded.drop(["Pop..Density", "Elect..Participtn..2009", "lat", "lon"], axis=1)
german_2009_last = german_2009_encoded.drop(["Elderly.pop.", "Elect..Particptn..2005", "lat", "lon"], axis=1)

In [37]:
german_2005_last= german_2005_last.rename(columns={"Elect..Particptn..2005": "Elect.Particip"})
german_2009_last= german_2009_last.rename(columns={"Elect..Participtn..2009": "Elect.Particip"})

In [44]:
german_2009_last.columns == german_2005_last.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [45]:
# save those versions for a second test on the model (2 files separated, no train on one year and prediction on another one)
german_2005_last.to_csv("data_new/german_2005_modif2.txt", index=False, sep=",")
german_2009_last.to_csv("data_new/german_2009_modif2.txt", index=False, sep=",")